<a href="https://colab.research.google.com/github/mori8/NLP-Pytorch-practice/blob/main/pytorch_lecture/Chapter_13_RNN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [13]:
HIDDEN_SIZE = 100
N_CHARS = 128
N_CLASSES = 18

In [2]:
class RNNClassifier(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers=1):
    super(RNNClassifier, self).__init__()
    self.hidden_size = hidden_size
    self.n_layers = n_layers
    # https://wikidocs.net/64779
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
    self.fc = nn.Linear(hidden_size, output_size)
  
  def forward(self, input):
    # input = B x S. size(0) = B
    batch_size = input.size(0)

    # input: B x S -- (transpose) --> S x B
    input = input.t()

    # Embedding S x B -> S x B x I (embedding size)
    print(" input", input.size())
    embedded = self.embedding(input)
    print(" embedding", embedded.size())

    # Make a hidden
    hidden = self._init_hidden(batch_size)
    output, hidden = self.gru(embedded, hidden)
    print(" gru hidden output", hidden.size())
    # GRU의 "마지막 cell의 hidden output"이 fc의 input
    fc_output = self.fc(hidden)
    # GRU의 output은 모든 cell의 output을 담고 있음
    print(" fc output", fc_output.size())
    return fc_output
  
  def _init_hidden(self, batch_size):
    hidden = torch.zeros(self.n_layers, batch_size, self.hidden_size)
    return Variable(hidden)

In [9]:
def str2ascii_arr(name):
  arr = [ord(c) for c in name]
  return arr, len(arr)

In [16]:
# Zero Padding
# 각 시퀀스와 가장 긴 시퀀스 길이의 차이만큼 생기는 공백을 0으로 채워 단어의 끝을 의미하도록 하는 방법
# 'adylov' = [97, 100, 121, 108, 111, 118], 'san' = [115, 97, 110]
# => 'adylov' = [97, 100, 121, 108, 111, 118], 'san' = [115, 97, 110, 0, 0, 0]
# Zero Padding이 된 vector를 임베딩해서 사용

def pad_sequences(vectorized_seqs, seq_lengths):
  seq_tensor = torch.zeros((len(vectorized_seqs), seq_lengths.max())).long()
  for idx, (seq, seq_len) in enumerate(zip(vectorized_seqs, seq_lengths)):
    seq_tensor[idx, :seq_len] = torch.LongTensor(seq)
  return seq_tensor

In [17]:
# Create necessary variables, lengths, and target
def make_variables(names):
  sequence_and_length = [str2ascii_arr(name) for name in names]
  vectorized_seqs = [sl[0] for sl in sequence_and_length]
  seq_lengths = torch.LongTensor([sl[1] for sl in sequence_and_length])
  return pad_sequences(vectorized_seqs, seq_lengths)

In [19]:
# RNN Classification

if __name__ == '__main__':
  names = ['adylov', 'solan', 'harb', 'san']
  classifier = RNNClassifier(N_CHARS, HIDDEN_SIZE, N_CLASSES)

  # for name in names:
  #   arr, _ = str2ascii_arr(name)
  #   inp = Variable(torch.LongTensor([arr]))
  #   out = classifier(inp)
  #   print("in", inp.size(), "outM", out.size())
  #   # in torch.Size([1, 6]) out torch.Size([1, 1, 18])(분류 결과)
  #   # in torch.Size([1, 5]) out torch.Size([1, 1, 18])

  inputs = make_variables(names)
  out = classifier(inputs)
  print("batch in", inputs.size(), "batch out", out.size())

 input torch.Size([6, 4])
 embedding torch.Size([6, 4, 100])
 gru hidden output torch.Size([1, 4, 100])
 fc output torch.Size([1, 4, 18])
batch in torch.Size([4, 6]) batch out torch.Size([1, 4, 18])


- packed sequence: https://github.com/hunkim/PyTorchZeroToAll/blob/master/13_4_pack_pad.py
- 참고: https://simonjisu.github.io/nlp/2018/07/05/packedsequence.html